In [1]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f

In [2]:
from pyspark.sql import Row
import datetime

l = [(datetime.date(2018,1,3), 'Ankit','F',25),
     (datetime.date(2018,2,3), 'Jalfaizy','F',22),
     (datetime.date(2018,1,5), 'saurabh','F',20),
     (datetime.date(2018,1,12), 'Bala','F',26),
     (datetime.date(2018,7,9), 'Jules','F',19) ,
     (datetime.date(2018,3,18), 'Arild','G',43),
     (datetime.date(2018,1,5), 'sarah','G',20),
     (datetime.date(2018,8,12), 'Boly','G',33),
     (datetime.date(2018,4,6), 'Anita','G',35),
     (datetime.date(2018,12,6), 'Jules','G',22),
     (datetime.date(2018,7,24), 'Soul','F',20),
     (datetime.date(2018,6,17), 'Gral','F',54),
     (datetime.date(2018,9,7), 'Apoh','F',18),
     (datetime.date(2018,10,4), 'Dony','F',32),
     (datetime.date(2018,2,5), 'Tanoh','F',31),
     (datetime.date(2018,11,12), 'Issouf','G',27),
     (datetime.date(2018,10,3), 'Bilé','G',29),
     (datetime.date(2018,5,3), 'Gagnon','G',20),
     (datetime.date(2018,3,5), 'Papiss','G',28),
     (datetime.date(2018,2,12), 'Kravitz','G',34),
     (datetime.date(2018,5,9), 'Mouli','F',35),
     (datetime.date(2018,8,3), 'Jacques','F',27),
     (datetime.date(2018,12,5), 'soum','F',22),
     (datetime.date(2018,4,12), 'MBra','F',36)]

rdd = spark_session.sparkContext.parallelize(l)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], sexe=x[2], age=int(x[3])))
schemaPeople = spark_session.createDataFrame(people)
schemaPeople.toPandas()

,age,date,name,sexe
0,25,2018-01-03,Ankit,F
1,22,2018-02-03,Jalfaizy,F
2,20,2018-01-05,saurabh,F
3,26,2018-01-12,Bala,F
4,19,2018-07-09,Jules,F
5,43,2018-03-18,Arild,G
6,20,2018-01-05,sarah,G
7,33,2018-08-12,Boly,G
8,35,2018-04-06,Anita,G
9,22,2018-12-06,Jules,G


1- compter le nombre de personne totale 
2- compter le nombre de fille et de garcon
3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)
4 - quelle est le nombre de jour écoulé depuis la dernière visite de chaque client par rapport à la date d'aujourdhui
5 - quels sont les personnes qui ont une date de visite < 400 jours.

#### 1- compter le nombre de personne totale 

In [3]:
#Nombre totale de personnes
print("Nombre total de personnes : {}".format(schemaPeople.count()))

Nombre total de personnes : 24


#### 2- compter le nombre de fille et de garcon

In [25]:
#Nombre de personnes par sexe
#print("Nombre totale de filles : {}".format(people.filter(lambda x: x[3] == 'F').count()))
#print("Nombre totale de garcons : {}".format(people.filter(lambda x: x[3] == 'G').count()))
schemaPeople.groupby('sexe').count().show()

+----+-----+
|sexe|count|
+----+-----+
|   F|   14|
|   G|   10|
+----+-----+



#### 3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)

In [70]:
other = schemaPeople.groupby('sexe').agg(f.min('age').alias('age min'),
                                 f.max('age').alias('age max'),
                                 f.count('age').alias('nb_pers'),
                                 f.mean('age').alias('age moyen')
                                 )

In [72]:
medi = [('G',schemaPeople.filter(schemaPeople['sexe'] == 'G').approxQuantile('age',(0.5,),0.001)[0],),
('F',schemaPeople.filter(schemaPeople['sexe'] == 'F').approxQuantile('age',(0.5,),0.001)[0],)]

rdd = spark_session.sparkContext.parallelize(medi)
med = rdd.map(lambda x: Row(sexe = x[0], mediane=x[1]))
schemaMedian = spark_session.createDataFrame(med)

In [71]:
schemaMedian.join(other, on = 'sexe').show()

+----+-------+-------+-------+-------+------------------+
|sexe|mediane|age min|age max|nb_pers|         age moyen|
+----+-------+-------+-------+-------+------------------+
|   F|   25.0|     18|     54|     14|27.642857142857142|
|   G|   28.0|     20|     43|     10|              29.1|
+----+-------+-------+-------+-------+------------------+



#### 4 - quelle est le nombre de jour écoulé depuis la dernière visite de chaque client par rapport à la date d'aujourdhui

In [98]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
ddl = dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit'))

ddl.toPandas()

,age,date,name,sexe,date_max,days_since_last_visit
0,25,2018-01-03,Ankit,F,2019-10-30,665
1,22,2018-02-03,Jalfaizy,F,2019-10-30,634
2,20,2018-01-05,saurabh,F,2019-10-30,663
3,26,2018-01-12,Bala,F,2019-10-30,656
4,19,2018-07-09,Jules,F,2019-10-30,478
5,43,2018-03-18,Arild,G,2019-10-30,591
6,20,2018-01-05,sarah,G,2019-10-30,663
7,33,2018-08-12,Boly,G,2019-10-30,444
8,35,2018-04-06,Anita,G,2019-10-30,572
9,22,2018-12-06,Jules,G,2019-10-30,328


#### 5 - quels sont les personnes qui ont une date de visite < 400 jours.

In [97]:
ddl.filter(ddl['days_since_last_visit'] < 400).show()

+---+----------+------+----+----------+---------------------+
|age|      date|  name|sexe|  date_max|days_since_last_visit|
+---+----------+------+----+----------+---------------------+
| 22|2018-12-06| Jules|   G|2019-10-30|                  328|
| 32|2018-10-04|  Dony|   F|2019-10-30|                  391|
| 27|2018-11-12|Issouf|   G|2019-10-30|                  352|
| 29|2018-10-03|  Bilé|   G|2019-10-30|                  392|
| 22|2018-12-05|  soum|   F|2019-10-30|                  329|
+---+----------+------+----+----------+---------------------+



#### 6- verifier s'il existe des valeurs manquantes

In [107]:
ddl.select([f.count(f.when(f.isnan(c), c)).alias("missing_"+ c)
                    for c in ddl.columns if c not in ["date", "date_max"] ]).show()

+-----------+------------+------------+-----------------------------+
|missing_age|missing_name|missing_sexe|missing_days_since_last_visit|
+-----------+------------+------------+-----------------------------+
|          0|           0|           0|                            0|
+-----------+------------+------------+-----------------------------+



#### 7-rajouter une colonne mois correspondant au mois de chaque individu dans le dataframe

In [100]:
df = schemaPeople.select("*", f.month('date').alias('mois'))
df.toPandas()

,age,date,name,sexe,mois
0,25,2018-01-03,Ankit,F,1
1,22,2018-02-03,Jalfaizy,F,2
2,20,2018-01-05,saurabh,F,1
3,26,2018-01-12,Bala,F,1
4,19,2018-07-09,Jules,F,7
5,43,2018-03-18,Arild,G,3
6,20,2018-01-05,sarah,G,1
7,33,2018-08-12,Boly,G,8
8,35,2018-04-06,Anita,G,4
9,22,2018-12-06,Jules,G,12


#### 8- Créer une colonne adulte qui prendra la valeur 0 pour les personnes de moins de 25 ans et 1 dans le cas contraire

In [108]:
df.withColumn("adulte", f.when(df["age"] < 25, 0).when(df["age"] >= 25,1)).toPandas()

,age,date,name,sexe,mois,adulte
0,25,2018-01-03,Ankit,F,1,1
1,22,2018-02-03,Jalfaizy,F,2,0
2,20,2018-01-05,saurabh,F,1,0
3,26,2018-01-12,Bala,F,1,1
4,19,2018-07-09,Jules,F,7,0
5,43,2018-03-18,Arild,G,3,1
6,20,2018-01-05,sarah,G,1,0
7,33,2018-08-12,Boly,G,8,1
8,35,2018-04-06,Anita,G,4,1
9,22,2018-12-06,Jules,G,12,0
